# Übung 3: 2. Textklassifikation

## Import Packages

In [67]:
!pip install pandas numpy wget matplotlib sklearn progressbar2
import os
import wget
import tarfile
import numpy as np
import pandas as pd
import re
import sklearn
import matplotlib.pyplot as plt
from typing import List, Tuple
from pathlib import Path
import progressbar
%matplotlib inline

Defaulting to user installation because normal site-packages is not writeableWARNING: pip is being invoked by an old script wrapper. This will fail in a future version of pip.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.



In [17]:
#%reload_ext version_information
#%version_information numpy, pandas

## a) Download und Entpacken von "20 Newsgroups"

In [22]:
url = 'http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz'
allNewsGroups_tarfile = '20news-18828.tar.gz'
allNewsGroups_folder = os.path.join(os.getcwd(), '20news-18828')

# Download file
if not os.path.isfile(allNewsGroups_tarfile):
    print("Downloading file...\n")

    wget.download(url, allNewsGroups_tarfile)
    dateDownloaded = !date #Calling Linux
    print("File downloaded on:", dateDownloaded)
else:
      print("Folder already downloaded!\n")

# Extract file
if not os.path.isdir(allNewsGroups_folder):
    try:
        os.mkdir(allNewsGroups_folder)
        print("Extracting...")
        tar = tarfile.open(allNewsGroups_tarfile, "r:gz")
        tar.extractall()
        print("Files extracted")
        tar.close()
    except:
        print("Extraction failed! Something went wrong.")
else:
    print("Folder already extracted!")

Folder already downloaded!

Folder already extracted!


## b) Einlesen der 4 Newsgroups

In [23]:
def strip_header(text):
    _before , _blankline , after = text.partition('\n\n')
    after.replace('\n',' ')
    return after


In [48]:
path: Path = Path(allNewsGroups_folder)  
folder_path = "c:\\Users\Andreas\\Documents\\GitHub\\ML-DL_repo\\uebungen\\uebung03\\20news-18828\\"
categories = ['alt.atheism','comp.graphics', 'sci.space', 'talk.religion.misc']
filesurvey = []

for newsgroup in categories:
    for row in os.walk(os.path.join(path, newsgroup)):   # row beinhaltet jeweils einen Ordnerinhalt
        for filename in row[2]:  # row[2] ist ein tupel aus Dateinamen
            full_path: Path = Path(row[0]) / Path(filename)   # row[0] ist der Ordnerpfad
            with open(full_path, 'r', encoding="ISO-8859-1") as file:
                data = strip_header(file.read())
            filesurvey.append([newsgroup, filename, data]) 
                    # full_path.stat().st_mtime, full_path.stat().st_size])


# raw_data = []
# labels = []

# for newsgroup in ["alt.atheism", "comp.graphics", "sci.space", "talk.religion.misc"]:
#     for subdir, _, files in os.walk(os.path.join(newsgroup20_folder, newsgroup)):
#         for file in files:
#             labels.append(newsgroup)
#             with open(os.path.join(subdir, file), 'r', encoding="ISO-8859-1") as f:
#                 s = strip_header(f.read())
#                 raw_data.append(s)

# print("Number of vectors:", len(raw_data))
# print("Number of labels:", len(labels))

## c) String in Worte (Tokens) zerlegen und Merkmalsvektorberechnung

In [52]:
l = []  # filesurvey[1][2]  [[:],[2]]
for i in range(len(filesurvey)):
    stripped_data = strip_header(filesurvey[i][2])
    l.extend(re.compile(r"(?u)\b[a-zA-Z]+\b").findall(stripped_data.lower()))

In [53]:
unique_tokens = []
for tokens in l:
    unique_tokens.extend(tokens)

unique_tokens = list(set(unique_tokens))
print("Number of unique tokens: {}".format(len(unique_tokens)))

Number of unique tokens: 26


In [65]:
FEATURE_VETORS_PATH = os.path.join(os.getcwd(), 'feature_vectors.npy')
TOKENS_PATH = os.path.join(os.getcwd(), 'tokens.npy')

def generate_feature_vector(_data, _tokens):
    _feature_vectors = np.zeros(shape=(len(_data), len(_tokens)))

    with progressbar.ProgressBar(max_value=len(_data)) as bar:
        for i, text in enumerate(_data):
            for j, token in enumerate(_tokens):
                _feature_vectors[i][j] = text.count(token)
            bar.update(i)

    _feature_vectors = np.asarray(_feature_vectors)
    np.save(FEATURE_VETORS_PATH, _feature_vectors)
    np.save(TOKENS_PATH, _tokens)

    return _feature_vectors

In [70]:
try:
    feature_vectors = np.load(FEATURE_VETORS_PATH)
    unique_tokens = np.load(TOKENS_PATH)
    print("Loading saved feature vectors.")
except IOError:
    print("No saved feature vectors found. Generating new ones..")
    feature_vectors = generate_feature_vector(l, unique_tokens)

No saved feature vectors found. Generating new ones..


TypeError: __init__() got an unexpected keyword argument 'max_value'